In [ ]:
!pip install langchain_community -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from datetime import datetime
import json

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

csv_path = "/content/interest_metrics.csv"
interest_data = pd.read_csv(csv_path)
interest_classes = []
for index, row in interest_data.iterrows():
    interest_classes.append(
        f"{row['Score']}: {row['Classification']}, {row['Description']}"
    )
interest_descriptions = "\n".join(interest_classes)


def interest_classification_function(transcription):
    chat_session = model.start_chat(history=[])
    prompt = (
        f"You are an AI assistant trained to analyze customer interactions and classify queries based on interest. "
        f"The interest is categorized into 10 classes. Here are the class definitions:\n\n"
        f"{interest_descriptions}\n\n"
        f"Your task is to analyze the following conversation, identify the customer's core query or concern, and classify it into one of these 10 interest classes. "
        f"Output only the class number corresponding to the interest level.\n\n"
        f"Conversation:\n{transcription}\n\n"
        f"Output:"
    )
    response = chat_session.send_message(prompt)
    return response.text.strip()


def requirment_extraction_function(transcription):
    chat_session = model.start_chat(history=[])
    prompt = (
        f"You are an AI assistant skilled in analyzing call-center dialogues. Your task is to extract the customer’s requirement from the conversation and summarize it concisely. "
        f"Focus on the core need or request of the customer and make the requirement short and insightful, highlighting what the customer wants so we can determine how to assist them. "
        f"Do not include any summaries or extra explanations, just provide the requirement.\n\n"
        f"Conversation:\n{transcription}\n\n"
        f"Output:"
    )
    response = chat_session.send_message(prompt)
    return response.text


def query_summarization_function(transcription):
    chat_session = model.start_chat(history=[])
    prompt = (
        f"You are a highly skilled AI assistant specializing in analyzing and summarizing call-center dialogues. Your role is to precisely identify and summarize the customer's primary query or concern. "
        f"The conversation transcript between the customer and the agent is provided below in dialogue format (e.g., Customer: ..., Agent: ...). Carefully analyze the conversation, focusing on key details, and generate a clear, concise summary of the customer's main issue or request. "
        f"Avoid generalizations and ensure the summary captures the specific query or concern expressed by the customer.\n\n"
        f"Conversation:\n{transcription}\n\n"
        f"Output Format:\n"
        f"The customer is inquiring about [specific query or concern].\n\n"
        f"Output:"
    )
    response = chat_session.send_message(prompt)
    return response.text



In [ ]:
def customer_info_extraction_function(transcription):
    chat_session = model.start_chat(history=[])
    prompt = (
        f"You are an AI assistant designed to extract customer's personal information from call center conversations. "
        f"Your task is to identify the customer's name, phone number, email address, place they are calling from, and any other relevant personal details like their address or date of birth. "
        f"Avoid extracting any work-related or requirements-related information. "
        f"If any of this personal information is not explicitly mentioned in the conversation, simply respond with 'Not Found' for that specific field. "
        f"\n\nConversation:\n{transcription}\n\n"
        f"Output Format:\n"
        f"{{'name': '[Customer Name]', 'customer_personal_info': {{'phone_number': '[Phone Number]', 'email': '[Email Address]', 'place': '[Place]', 'other_info': '[Other Personal Information]'}}}}\n\n"  # Nested structure
        f"Output:"
    )
    response = chat_session.send_message(prompt)
    import json
    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        print(f"Warning: Could not decode JSON response: {response.text}")
        # Return default values with the desired structure
        return {"name": "Not Found", "customer_personal_info": {"phone_number": "Not Found", "email": "Not Found", "place": "Not Found", "other_info": "Not Found"}}

In [ ]:

transcription_file_path = "/content/transcription.txt"
with open(transcription_file_path, "r") as file:
    transcription = file.read()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from datetime import datetime

interest_chain = RunnableParallel(interest_classification=interest_classification_function)
requirment_extraction_chain = RunnableParallel(requirment_extraction=requirment_extraction_function)
query_summarization_chain = RunnableParallel(query_summarization=query_summarization_function)

combined_chain = RunnableParallel(
    interest=interest_chain,
    requirment=requirment_extraction_chain,
    summary=query_summarization_chain,
    customer_info=customer_info_extraction_function,
)

results = combined_chain.invoke({"transcription": transcription})
output = {
    "id": "unique_id",
    "summary": results['summary']['query_summarization'],
    "requirment": results['requirment']['requirment_extraction'],
    "InterestLVL": results['interest']['interest_classification'],
    "Date": datetime.now().strftime("%Y-%m-%d"),
    "name": results['customer_info']['name'],  # Customer's name
    "customer_personal_info": {  # Other personal information
        "phone_number": results['customer_info']['customer_personal_info']['phone_number'],
        "email": results['customer_info']['customer_personal_info']['email'],
        "place": results['customer_info']['customer_personal_info']['place'],
        "other_info": results['customer_info']['customer_personal_info']['other_info']
    }
}

{'name': 'Mr. Armando', 'customer_personal_info': {'phone_number': 'Not Found', 'email': 'Not Found', 'place': 'Not Found', 'other_info': 'Not Found'}}
```



In [ ]:

print(output)

{'id': 'unique_id', 'summary': 'The customer is inquiring about renting a car from the San José del Cabo International Airport (SJD) from the 21st to the 25th, ideally a Jeep Wrangler or similar SUV, and needs assistance determining pricing and availability from different rental car companies partnered with World Discovery Pacific.\n', 'query': 'The customer requested to rent a car from the airport, specifying pick-up and drop-off dates and times, and inquired about the availability of specific car types and pricing, including taxes.\n', 'urgencylvl': '5', 'elevation': 'yes', 'Date': '2024-12-14', 'name': 'Not Found', 'customer_personal_info': {'phone_number': 'Not Found', 'email': 'Not Found', 'place': 'Not Found', 'other_info': 'Not Found'}}
